<a href="https://colab.research.google.com/github/HanhengHe/BigDataCourseProj/blob/main/bigdataproj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# GET FILE

from google.colab import drive
import zipfile

drive.mount('/content/drive/', force_remount=False)

with zipfile.ZipFile("/content/drive/MyDrive/Dataset_small/data1.zip","r") as zip_ref:
    zip_ref.extractall("targetdir")

Mounted at /content/drive/


In [3]:
# NETWORK DEFINE MODULE

import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
import torch.nn.functional as F

torch.cuda._initialized = True

class Inception(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1_b1 = nn.Conv2d(256, 256, 1)
        self.conv2_b1 = nn.Conv2d(256, 320, 3, padding=1)
        self.conv3_b1 = nn.Conv2d(320, 320, 3, padding=1)

        self.conv1_b2 = nn.Conv2d(256, 48, 1)
        self.conv2_b2 = nn.Conv2d(48, 64, (1, 5), padding=(0, 2))
        self.conv3_b2 = nn.Conv2d(64, 64, (5, 1), padding=(2, 0))

        self.conv1_b3 = nn.Conv2d(256, 64, 1)

        self.conv1_b4 = nn.Conv2d(256, 64, 1)

    def forward(self, X):

        # branch 1
        b1 = self.conv1_b1(X)
        b1 = b1.mul(torch.sigmoid(b1))

        b1 = self.conv2_b1(b1)
        b1 = b1.mul(torch.sigmoid(b1))

        b1 = self.conv3_b1(b1)
        b1 = b1.mul(torch.sigmoid(b1))

        # branch 2
        b2 = self.conv1_b2(X)
        b2 = b2.mul(torch.sigmoid(b2))

        b2 = self.conv2_b2(b2)
        b2 = b2.mul(torch.sigmoid(b2))

        b2 = self.conv3_b2(b2)
        b2 = b2.mul(torch.sigmoid(b2))

        # branch 3
        b3 = self.conv1_b3(X)
        b3 = b3.mul(torch.sigmoid(b3))

        # branch 4
        b4 = F.max_pool2d(X, (3, 3), stride=1, padding=1)
        b4 = self.conv1_b4(b4)
        b4 = b4.mul(torch.sigmoid(b4))

        return torch.cat([b1, b2, b3, b4], 1)


class LeNet_II(nn.Module):
    def __init__(self):
        super().__init__()

        # convolutional layer
        self.conv1_b1 = nn.Conv2d(3, 64, 3, padding=1)
        self.conv2_b1 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3_b1 = nn.Conv2d(128, 256, 3, padding=1)

        self.conv1_b2 = nn.Conv2d(3, 64, 3, padding=2, dilation=2)
        self.conv2_b2 = nn.Conv2d(64, 128, 3, padding=2, dilation=2)
        self.conv3_b2 = nn.Conv2d(128, 256, 3, padding=2, dilation=2)
        self.conv4_b2 = nn.Conv2d(256, 512, 3, padding=2, dilation=2)

        self.conv = nn.Conv2d(1024, 768, 3, padding=1)

        # full connect layer
        self.fc1 = nn.Linear(768 * 4, 1024)
        self.fc2 = nn.Linear(1024, 3926)

        # Inception 
        self.Inception = Inception()

    # forward propagation
    def forward(self, X):
        # branch 1
        b1 = self.conv1_b1(X)
        b1 = F.max_pool2d(b1, (2, 2), stride=2)
        b1 = b1.mul(torch.sigmoid(b1))

        b1 = self.conv2_b1(b1)
        b1 = F.max_pool2d(b1, (2, 2), stride=2)
        b1 = b1.mul(torch.sigmoid(b1))

        b1 = self.conv3_b1(b1)
        b1 = F.max_pool2d(b1, (2, 2), stride=2)
        b1 = b1.mul(torch.sigmoid(b1))

        b1 = self.Inception(b1)

        # branch 2
        b2 = self.conv1_b2(X)
        b2 = F.max_pool2d(b2, (2, 2), stride=2)
        b2 = b2.mul(torch.sigmoid(b2))

        b2 = self.conv2_b2(b2)
        b2 = F.max_pool2d(b2, (2, 2), stride=2)
        b2 = b2.mul(torch.sigmoid(b2))

        b2 = self.conv3_b2(b2)
        b2 = b2.mul(torch.sigmoid(b2))

        b2 = self.conv4_b2(b2)
        b2 = F.max_pool2d(b2, (2, 2), stride=2)
        b2 = b2.mul(torch.sigmoid(b2))

        # concatenate
        out = self.conv(torch.cat([b1, b2], 1))
        out = F.max_pool2d(out, (2, 2), stride=2)
        out = out.mul(torch.sigmoid(out))

        # unfold
        out = torch.flatten(out, start_dim=1)

        # full connect
        out = self.fc1(out)
        out = out.mul(torch.sigmoid(out))
        out = F.dropout(out)

        out = self.fc2(out)
        out = F.log_softmax(out, dim=-1)

        return out

In [17]:
# LOADING TRAINING DATA

lsTrainLabel = []
lsTrainData = []

with open("/content/targetdir/data1/train_label.txt") as f:
  for line in f.readlines():
    expression = line.split(' ')
    lsTrainLabel.append(int(expression[1]))
    dataRGB = np.load("/content/targetdir/data1/train/" + expression[0]).transpose((2,0,1))
    lsTrainData.append(dataRGB)
    # turn training data into grayscale data
    # lsData.append((dataRGB[:, :, 0] + dataRGB[:, :, 1] + dataRGB[:, :, 2]) / 3)

print("Number of traning data: {0}".format(len(lsTrainLabel)), end='\n')
print("Shape of traning data: {0}".format(lsTrainData[0].shape), end='\n')

Number of traning data: 50000
Shape of traning data: (3, 32, 32)


In [30]:
# TRAINING MODULE

import math

CPU = torch.device("cpu")
DEVICE = torch.device("cuda:0") if torch.cuda.is_available() else CPU
print("DEVICE = %s" % (DEVICE))
torch.cuda.empty_cache()

trainLabels = np.array(lsTrainLabel)
trainSet = np.array(lsTrainData)

trainSet = torch.from_numpy(trainSet).float()
trainLabels = torch.from_numpy(trainLabels).to(DEVICE)

MINIBATCH_SIZE = 512    # mini batch size

# first transform the data to dataset can be processed by torch
torch_dataset = Data.TensorDataset(trainSet, trainLabels.long())
# put the dataset into DataLoader
loader = Data.DataLoader(
    dataset=torch_dataset,
    batch_size=MINIBATCH_SIZE,
    shuffle=True,
    num_workers=0
)

model = LeNet_II().to(DEVICE)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters())

Epoch = 50
nSteps = math.ceil(trainLabels.shape[0] / MINIBATCH_SIZE)
MINLOSS = 0.01
outputFrequence = 10

model.train()
for epoch in range(Epoch):
    nCurLoss = 0
    for step, (batch_x, batch_y) in enumerate(loader):
        optimizer.zero_grad()        
        output = model(batch_x.to(DEVICE))
        loss = F.nll_loss(output, batch_y)
        loss.backward()
        optimizer.step()
        nCurLoss += loss.item() / nSteps
    if epoch == 0 or (epoch + 1) % outputFrequence == 0:
      print("Epoch {0}; General loss: {1}".format(epoch+1, nCurLoss))
    if float(nCurLoss) <= MINLOSS:
        break

DEVICE = cuda:0
Epoch 1; General loss: 7.376458609104158
Epoch 10; General loss: 1.5692575395107269
Epoch 20; General loss: 0.7500250339508058
Epoch 30; General loss: 0.0412509759888053


In [31]:
# LOAD TEST DATA

lsTestLabel = []
lsTestData = []

with open("/content/targetdir/data1/test_label.txt") as f:
  for line in f.readlines():
    expression = line.split(' ')
    lsTestLabel.append(int(expression[1]))
    dataRGB = np.load("/content/targetdir/data1/test/" + expression[0]).transpose((2,0,1))
    lsTestData.append(dataRGB)
    # turn training data into grayscale data
    # lsData.append((dataRGB[:, :, 0] + dataRGB[:, :, 1] + dataRGB[:, :, 2]) / 3)

print("Number of testing data: {0}".format(len(lsTestLabel)), end='\n')
print("Shape of testing data: {0}".format(lsTestData[0].shape), end='\n')

Number of testing data: 10000
Shape of testing data: (3, 32, 32)


In [32]:
# TEST MODULE

testLabels = np.array(lsTestLabel)
testSet = np.array(lsTestData)

testSet = torch.from_numpy(testSet).float()
testLabels = torch.from_numpy(testLabels).to(DEVICE)

model.eval()
test_loss = 0
correct = 0

# first transform the data to dataset can be processed by torch
torch_dataset = Data.TensorDataset(testSet, testLabels.long())
# put the dataset into DataLoader
loader = Data.DataLoader(
    dataset=torch_dataset,
    batch_size=MINIBATCH_SIZE,
    shuffle=True,
    num_workers=0
)

with torch.no_grad():
    for step, (batch_x, batch_y) in enumerate(loader):
        output = model(batch_x.to(DEVICE))
        test_loss += F.nll_loss(output, batch_y, reduction='sum').item()  # sum a batch of loss
        predict = output.max(1, keepdim=True)[1]  # find the prediction
        correct += predict.eq(batch_y.view_as(predict)).sum().item()

testSize = testLabels.shape[0]
test_loss /= testSize
print("Test: Average loss:%s, Accuracy: %s/%s (%s)"
      % (test_loss, correct, testSize, correct / testSize))

Test: Average loss:0.0055739856243133545, Accuracy: 9990/10000 (0.999)
